In [25]:
import numpy as np
from scipy.optimize import differential_evolution, root_scalar, minimize
from openpyxl import load_workbook
from consts_and_funcs import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Circle

# 设置 matplotlib 使用支持中文的字体
plt.rcParams['font.sans-serif'] = ['SimHei'] # 使用黑体
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题

# Global list to record optimization history
optimization_history = []

# Function to calculate shielding time
def my_shield_time(theta1, v1, t_deploy1, delay1, theta2, v2, t_deploy2, delay2, theta3, v3, t_deploy3, delay3, record_intervals=False):
    # Drone directions and velocities
    dir_unit1 = np.array([-np.cos(theta1), np.sin(theta1), 0])
    drone_vel1 = v1 * dir_unit1
    dir_unit2 = np.array([-np.cos(theta2), np.sin(theta2), 0])
    drone_vel2 = v2 * dir_unit2
    dir_unit3 = np.array([-np.cos(theta3), np.sin(theta3), 0])
    drone_vel3 = v3 * dir_unit3

    # Deployment and explosion positions for each bomb
    deploy_positions = []
    explode_positions = []
    explode_times = []
    # FY1
    deploy_pos1 = FY1_INIT + drone_vel1 * t_deploy1
    explode_pos1 = deploy_pos1 + drone_vel1 * delay1 + np.array([0, 0, -0.5 * G * delay1**2])
    explode_t1 = t_deploy1 + delay1
    deploy_positions.append(deploy_pos1)
    explode_positions.append(explode_pos1)
    explode_times.append(explode_t1)
    # FY2
    deploy_pos2 = FY2_INIT + drone_vel2 * t_deploy2
    explode_pos2 = deploy_pos2 + drone_vel2 * delay2 + np.array([0, 0, -0.5 * G * delay2**2])
    explode_t2 = t_deploy2 + delay2
    deploy_positions.append(deploy_pos2)
    explode_positions.append(explode_pos2)
    explode_times.append(explode_t2)
    # FY3
    deploy_pos3 = FY3_INIT + drone_vel3 * t_deploy3
    explode_pos3 = deploy_pos3 + drone_vel3 * delay3 + np.array([0, 0, -0.5 * G * delay3**2])
    explode_t3 = t_deploy3 + delay3
    deploy_positions.append(deploy_pos3)
    explode_positions.append(explode_pos3)
    explode_times.append(explode_t3)

    # Cloud center function for each bomb
    def get_cloud_center(i):
        def cloud_center(t):
            tau = t - explode_times[i]
            if tau < 0:
                return explode_positions[i] + np.array([0, 0, -ESCAPE_VEL * tau])
            if tau > DURATION:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
            else:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * tau])
        return cloud_center

    # Distance difference function
    def min_distance_diff(t, cylinder_radius):
        min_distance = []
        for i in range(3):
            cloud_center = get_cloud_center(i)
            min_distance.append(get_distance_diff(cloud_center, cylinder_radius, 1)(t))
        return min(min_distance)

    # Calculate total shielding time for a given radius
    def calculate_shield_time(cylinder_radius, record_intervals=False):
        def diff(t):
            return min_distance_diff(t, cylinder_radius)
            t_max = M1_TIME
t = min(explode_times)
total_time = 0.0
intervals = []
while t < t_max:
    while t < t_max and diff(t) > 0:
        t += 0.1
    if t >= t_max:
        break
    t_start = t
    while t < t_max and diff(t) <= 0:
        t += 0.1
    t_end = t
    if t_start != min(explode_times):
        root1 = root_scalar(diff, bracket=[t_start - 0.1, t_start], method='brentq')
        t1 = root1.root if root1.converged else t_start
    else:
        t1 = min(explode_times)
    if t_end < t_max:
        root2 = root_scalar(diff, bracket=[t_start, t_end], method='brentq')
        t2 = root2.root if root2.converged else t_end - 0.1
    else:
        t2 = t_max
    total_time += max(0, t2 - t1)
    if record_intervals:
        intervals.append((t1, t2))
if record_intervals:
    return total_time, intervals
return total_time

inner = calculate_shield_time(INNER_RADIUS)
outer = calculate_shield_time(OUTER_RADIUS)
avg_time = (inner + outer) / 2

if record_intervals:
    _, inner_intervals = calculate_shield_time(INNER_RADIUS, True)
    _, outer_intervals = calculate_shield_time(OUTER_RADIUS, True)
    return avg_time, inner_intervals

return avg_time
# 计算偏心的屏蔽时间函数
def par_shield_time(theta, v, t_deploy, delay, par_id):
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    drone_init = [FY1_INIT, FY2_INIT, FY3_INIT][par_id - 1]
    
    deploy_pos = drone_init + drone_vel * t_deploy
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    calculate_shield_time = get_calculate_shield_time(cloud_center, explode_t, 1)
    inner = calculate_shield_time(INNER_RADIUS)
    outer = calculate_shield_time(OUTER_RADIUS)
    return (inner + outer) / 2

def my_min_distance(theta1, v1, t_deploy1, delay1, theta2, v2, t_deploy2, delay2, theta3, v3, t_deploy3, delay3):
    dir_unit1 = np.array([-np.cos(theta1), np.sin(theta1), 0])
    drone_vel1 = v1 * dir_unit1
    dir_unit2 = np.array([-np.cos(theta2), np.sin(theta2), 0])
    drone_vel2 = v2 * dir_unit2
    dir_unit3 = np.array([-np.cos(theta3), np.sin(theta3), 0])
    drone_vel3 = v3 * dir_unit3
    
    deploy_positions = []
    explode_positions = []
    explode_times = []
    deploy_pos1 = FY1_INIT + drone_vel1 * t_deploy1
    explode_pos1 = deploy_pos1 + drone_vel1 * delay1 + np.array([0, 0, -0.5 * G * delay1**2])
    explode_t1 = t_deploy1 + delay1
    deploy_positions.append(deploy_pos1)
    explode_positions.append(explode_pos1)
    explode_times.append(explode_t1)
    deploy_pos2 = FY2_INIT + drone_vel2 * t_deploy2
    explode_pos2 = deploy_pos2 + drone_vel2 * delay2 + np.array([0, 0, -0.5 * G * delay2**2])
    explode_t2 = t_deploy2 + delay2
    deploy_positions.append(deploy_pos2)
    explode_positions.append(explode_pos2)
    explode_times.append(explode_t2)
    deploy_pos3 = FY3_INIT + drone_vel3 * t_deploy3
    explode_pos3 = deploy_pos3 + drone_vel3 * delay3 + np.array([0, 0, -0.5 * G * delay3**2])
    explode_t3 = t_deploy3 + delay3
    deploy_positions.append(deploy_pos3)
    explode_positions.append(explode_pos3)
    explode_times.append(explode_t3)
    
    def get_cloud_center(i):
        def cloud_center(t):
            tau = t - explode_times[i]
            if tau < 0:
                return explode_positions[i] + np.array([0, 0, -ESCAPE_VEL * tau])
            if tau > DURATION:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
            else:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * tau])
        return cloud_center
    
    def min_distance_diff_rough(t):
        min_distance = []
        for i in range(3):
            cloud_center = get_cloud_center(i)
            min_distance.append(get_distance_diff_rough(cloud_center, 1)(t))
        return min(min_distance)
    
    def min_distance(explode_t):
        t_max = M1_TIME
        if t_max - explode_t <= 0.1:
            return min_distance_diff_rough(min(t_max, explode_t))
        t_arange = np.arange(explode_t, t_max, 0.1)
        distance_arange = np.array([min_distance_diff_rough(t) for t in t_arange])
        return distance_arange.min()
    return min_distance(min(explode_times))
    def par_min_distance(theta, v, t_deploy, delay, par_id):
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    drone_init = [FY1_INIT, FY2_INIT, FY3_INIT][par_id - 1]
    
    deploy_pos = drone_init + drone_vel * t_deploy
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    return get_min_distance(cloud_center, explode_t, 1)

def single_bomb_shield_time(theta, v, t_deploy, delay, drone_num):
    if drone_num == 1:
        drone_init = FY1_INIT
    elif drone_num == 2:
        drone_init = FY2_INIT
    elif drone_num == 3:
        drone_init = FY3_INIT
    else:
        raise ValueError('drone_num只能从1到3！')
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    
    deploy_pos = drone_init + drone_vel * t_deploy
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    calculate_shield_time = get_calculate_shield_time(cloud_center, explode_t, 1)
    inner = calculate_shield_time(INNER_RADIUS)
    outer = calculate_shield_time(OUTER_RADIUS)
    return (inner + outer) / 2

def objective0(params):
    return my_min_distance(*params)

def get_objective0(par_id):
    def objective0(params):
        return par_min_distance(*params, par_id)
    return objective0

def objective(params):
    return -my_shield_time(*params)

def get_objective(par_id):
    def objective(params):
        return -par_shield_time(*params, par_id)
    return objective

if __name__ == '__main__':
    # Bounds for 3 drones: [theta1, v1, t_deploy1, delay1, theta2, v2, t_deploy2, delay2, theta3, v3, t_deploy3, delay3]
    bounds = [(-np.pi/6, np.pi/6), (70, 140), (0, 5), (0, 5),  # FY1
              (-2*np.pi/3, 0), (70, 140), (4, 10), (5, 10),  # FY2
              (np.pi/3, 2*np.pi/3), (70, 140), (20, 30), (0, 10)]  # FY3
    
    # Callback to monitor optimization progress
    iteration = 0
    def callback(xk, convergence):
        global iteration
        iteration += 1
        current_time = -objective(xk)
        optimization_history.append(current_time)  # Record every iteration
        log_message = (f"迭代 {iteration}: 最佳屏蔽时间 = {current_time:.3f} 秒, "
                      f"theta1 = {np.degrees(xk[0]) % 360:.3f}°, v1 = {xk[1]:.3f} m/s, "
                      f"t_deploy1 = {xk[2]:.3f} s, delay1 = {xk[3]:.3f} s\n")
        print(log_message.strip())
        with open('optimization_log4.txt', 'a', encoding='utf-8') as f:
            f.write(log_message)
    
    def get_callback(par_id):
        def callback(xk, convergence):
            global iteration
            iteration += 1
            current_time = -get_objective(par_id)(xk)
            log_message = (f"迭代 {iteration}: 最佳屏蔽时间 = {current_time:.3f} 秒, par_id = {par_id}, "
                          f"theta = {np.degrees(xk[0]) % 360:.3f}°, v = {xk[1]:.3f} m/s, "
                          f"t_deploy = {xk[2]:.3f} s, delay = {xk[3]:.3f} s\n")
            print(log_message.strip())
            with open('optimization_log4.txt', 'a', encoding='utf-8') as f:
                f.write(log_message)
        return callback
        # 使用 differential_evolution 优化最短距离
result0 = [[] for _ in range(3)]
for id in range(1, 4):
    for _ in range(20):
        a_result0 = differential_evolution(
            get_objective0(id),
            bounds[4*id - 4: 4*id],
            strategy='best1bin',
            maxiter=10,
            popsize=5,
            tol=1e-3,
            callback=get_callback(id),
            init='random',
            disp=False,
            polish=False
        )
        result0[id - 1].append(a_result0.x)

# 提取20个解作为第二步的初始种群
initial_population = np.array([[a, b, c] for a, b, c in zip(result0[0], result0[1], result0[2])])
result = differential_evolution(
    objective,
    bounds,
    strategy='best1bin',
    maxiter=200,
    popsize=20,
    tol=1e-3,
    callback=callback,
    init=initial_population,
    disp=False,
    polish=True
)

theta1, v1, t_deploy1, delay1, theta2, v2, t_deploy2, delay2, theta3, v3, t_deploy3, delay3 = result.x
opt_time, shielding_intervals = my_shield_time(theta1, v1, t_deploy1, delay1, theta2, v2, t_deploy2, delay2, theta3, v3, t_deploy3, delay3, record_intervals=True)

# Calculate positions
dir_unit1 = np.array([-np.cos(theta1), np.sin(theta1), 0])
drone_vel1 = v1 * dir_unit1
deploy_pos1 = FY1_INIT + drone_vel1 * t_deploy1
explode_pos1 = deploy_pos1 + drone_vel1 * delay1 + np.array([0, 0, -0.5 * G * delay1**2])

dir_unit2 = np.array([-np.cos(theta2), np.sin(theta2), 0])
drone_vel2 = v2 * dir_unit2
deploy_pos2 = FY2_INIT + drone_vel2 * t_deploy2
explode_pos2 = deploy_pos2 + drone_vel2 * delay2 + np.array([0, 0, -0.5 * G * delay2**2])

dir_unit3 = np.array([-np.cos(theta3), np.sin(theta3), 0])
drone_vel3 = v3 * dir_unit3
deploy_pos3 = FY3_INIT + drone_vel3 * t_deploy3
explode_pos3 = deploy_pos3 + drone_vel3 * delay3 + np.array([0, 0, -0.5 * G * delay3**2])

# Convert theta to degrees
theta_deg1 = np.degrees(theta1) % 360
theta_deg2 = np.degrees(theta2) % 360
theta_deg3 = np.degrees(theta3) % 360

# Prepare data for Excel
data = [
    {
        '无人机编号': 'FY1',
        '无人机运动方向': round(theta_deg1, 3),
        '无人机运动速度(m/s)': round(v1, 3),
        '烟幕干扰弹编号': 1,
        '烟幕干扰弹投放点的x坐标(m)': round(deploy_pos1[0], 3),
        '烟幕干扰弹投放点的y坐标(m)': round(deploy_pos1[1], 3),
        '烟幕干扰弹投放点的z坐标(m)': round(deploy_pos1[2], 3),
        '烟幕干扰弹起爆点的x坐标(m)': round(explode_pos1[0], 3),
        '烟幕干扰弹起爆点的y坐标(m)': round(explode_pos1[1], 3),
        '烟幕干扰弹起爆点的z坐标(m)': round(explode_pos1[2], 3),
        '有效干扰时长(s)': round(single_bomb_shield_time(theta1, v1, t_deploy1, delay1, 1), 3)
    },
    {
        '无人机编号': 'FY2',
        '无人机运动方向': round(theta_deg2, 3),
        '无人机运动速度(m/s)': round(v2, 3),
        '烟幕干扰弹编号': 2,
        '烟幕干扰弹投放点的x坐标(m)': round(deploy_pos2[0], 3),
        '烟幕干扰弹投放点的y坐标(m)': round(deploy_pos2[1], 3),
        '烟幕干扰弹投放点的z坐标(m)': round(deploy_pos2[2], 3),
        '烟幕干扰弹起爆点的x坐标(m)': round(explode_pos2[0], 3),
        '烟幕干扰弹起爆点的y坐标(m)': round(explode_pos2[1], 3),
        '烟幕干扰弹起爆点的z坐标(m)': round(explode_pos2[2], 3),
        '有效干扰时长(s)': round(single_bomb_shield_time(theta2, v2, t_deploy2, delay2, 2), 3)
    },
    {
        '无人机编号': 'FY3',
        '无人机运动方向': round(theta_deg3, 3),
        '无人机运动速度(m/s)': round(v3, 3),
        '烟幕干扰弹编号': 3,
        '烟幕干扰弹投放点的x坐标(m)': round(deploy_pos3[0], 3),
        '烟幕干扰弹投放点的y坐标(m)': round(deploy_pos3[1], 3),
        '烟幕干扰弹投放点的z坐标(m)': round(deploy_pos3[2], 3),
        '烟幕干扰弹起爆点的x坐标(m)': round(explode_pos3[0], 3),
        '烟幕干扰弹起爆点的y坐标(m)': round(explode_pos3[1], 3),
        '烟幕干扰弹起爆点的z坐标(m)': round(explode_pos3[2], 3),
        '有效干扰时长(s)': round(single_bomb_shield_time(theta3, v3, t_deploy3, delay3, 3), 3)
    }
]
file_path = 'A题/附件/result2.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active
for row in range(2, 5):
    theta = [theta1, theta2, theta3][row - 2]
    theta_deg = [theta_deg1, theta_deg2, theta_deg3][row - 2]
    opt_v = [v1, v2, v3][row - 2]
    sheet.cell(row=row, column=2).value = round(theta_deg, 3)
    sheet.cell(row=row, column=3).value = round(opt_v, 3)
    deploy_pos = [deploy_pos1, deploy_pos2, deploy_pos3][row - 2]
    explode_pos = [explode_pos1, explode_pos2, explode_pos3][row - 2]
    opt_t_deploy = [t_deploy1, t_deploy2, t_deploy3][row - 2]
    opt_delay = [delay1, delay2, delay3][row - 2]
    sheet.cell(row=row, column=4).value = round(deploy_pos[0], 3)
    sheet.cell(row=row, column=5).value = round(deploy_pos[1], 3)
    sheet.cell(row=row, column=6).value = round(deploy_pos[2], 3)
    sheet.cell(row=row, column=7).value = round(explode_pos[0], 3)
    sheet.cell(row=row, column=8).value = round(explode_pos[1], 3)
    sheet.cell(row=row, column=9).value = round(explode_pos[2], 3)
    sheet.cell(row=row, column=10).value = round(single_bomb_shield_time(theta, opt_v, opt_t_deploy, opt_delay, row - 1), 3)

workbook.save(file_path)
print(f"已将内容写入 {file_path}.")

# Output results
final_output = (
    f"优化结果：\n"
    f"总迭代次数: {result.nit}\n"
    f"无人机 FY1: 运动方向: {theta_deg1:.3f}°, 速度: {v1:.3f} m/s\n"
    f"干扰弹 1: 投放时间: {t_deploy1:.3f} s, 起爆延迟: {delay1:.3f} s\n"
    f" 投放位置: {deploy_pos1}\n"
    f" 起爆位置: {explode_pos1}\n"
    f" 遮蔽时间: {single_bomb_shield_time(theta1, v1, t_deploy1, delay1, 1):.3f} s\n"
    f"无人机 FY2: 运动方向: {theta_deg2:.3f}°, 速度: {v2:.3f} m/s\n"
    f"干扰弹 2: 投放时间: {t_deploy2:.3f} s, 起爆延迟: {delay2:.3f} s\n"
    f" 投放位置: {deploy_pos2}\n"
    f" 起爆位置: {explode_pos2}\n"
    f" 遮蔽时间: {single_bomb_shield_time(theta2, v2, t_deploy2, delay2, 2):.3f} s\n"
    f"无人机 FY3: 运动方向: {theta_deg3:.3f}°, 速度: {v3:.3f} m/s\n"
    f"干扰弹 3: 投放时间: {t_deploy3:.3f} s, 起爆延迟: {delay3:.3f} s\n"
    f" 投放位置: {deploy_pos3}\n"
    f" 起爆位置: {explode_pos3}\n"
    f" 遮蔽时间: {single_bomb_shield_time(theta3, v3, t_deploy3, delay3, 3):.3f} s\n"
    f"总有效干扰时长(平均): {opt_time:.3f} s\n"
)
print(final_output.strip())
with open('optimization_log4.txt', 'a', encoding='utf-8') as f:
    f.write(final_output + "\n")

# Visualization 1: Motion trajectories schematic (3D plot)
fig1 = plt.figure(figsize=(12, 9))
ax1 = fig1.add_subplot(111, projection='3d')

# Missile M1 trajectory
m1_init = np.array([20000, 0, 2000], dtype=np.float64)
direction_to_fake = np.array([0, 0, 0], dtype=np.float64) - m1_init
direction_to_fake /= np.linalg.norm(direction_to_fake)
missile_speed = 300
t_missile = np.linspace(0, M1_TIME, 100)
m1_traj = m1_init + t_missile[:, np.newaxis] * direction_to_fake * missile_speed
ax1.plot(m1_traj[:, 0], m1_traj[:, 1], m1_traj[:, 2], 'r-', label='导弹 M1 轨迹', linewidth=2)

# Drone trajectories
t_max_drone = max(t_deploy1 + delay1, t_deploy2 + delay2, t_deploy3 + delay3) + 5
t_drone = np.linspace(0, t_max_drone, 100)
fy1_traj = FY1_INIT + t_drone[:, np.newaxis] * drone_vel1
fy2_traj = FY2_INIT + t_drone[:, np.newaxis] * drone_vel2
fy3_traj = FY3_INIT + t_drone[:, np.newaxis] * drone_vel3
ax1.plot(fy1_traj[:, 0], fy1_traj[:, 1], fy1_traj[:, 2], 'b-', label='无人机 FY1 轨迹', linewidth=2)
ax1.plot(fy2_traj[:, 0], fy2_traj[:, 1], fy2_traj[:, 2], 'c-', label='无人机 FY2 轨迹', linewidth=2)
ax1.plot(fy3_traj[:, 0], fy3_traj[:, 1], fy3_traj[:, 2], 'm-', label='无人机 FY3 轨迹', linewidth=2)
# Bomb trajectories
times1 = np.linspace(t_deploy1, t_deploy1 + delay1, 20)
gravity_term1 = np.zeros((20, 3), dtype=np.float64)
gravity_term1[:, 2] = -0.5 * G * (times1 - t_deploy1)**2
bomb_traj1 = FY1_INIT + times1[:, np.newaxis] * drone_vel1 + gravity_term1
ax1.plot(bomb_traj1[:, 0], bomb_traj1[:, 1], bomb_traj1[:, 2], 'g--', label='干扰弹 1 轨迹', alpha=0.7)

times2 = np.linspace(t_deploy2, t_deploy2 + delay2, 20)
gravity_term2 = np.zeros((20, 3), dtype=np.float64)
gravity_term2[:, 2] = -0.5 * G * (times2 - t_deploy2)**2
bomb_traj2 = FY2_INIT + times2[:, np.newaxis] * drone_vel2 + gravity_term2
ax1.plot(bomb_traj2[:, 0], bomb_traj2[:, 1], bomb_traj2[:, 2], 'g--', label='干扰弹 2 轨迹', alpha=0.7)

times3 = np.linspace(t_deploy3, t_deploy3 + delay3, 20)
gravity_term3 = np.zeros((20, 3), dtype=np.float64)
gravity_term3[:, 2] = -0.5 * G * (times3 - t_deploy3)**2
bomb_traj3 = FY3_INIT + times3[:, np.newaxis] * drone_vel3 + gravity_term3
ax1.plot(bomb_traj3[:, 0], bomb_traj3[:, 1], bomb_traj3[:, 2], 'g--', label='干扰弹 3 轨迹', alpha=0.7)

# Explosion points
ax1.scatter([explode_pos1[0], explode_pos2[0], explode_pos3[0]],
            [explode_pos1[1], explode_pos2[1], explode_pos3[1]],
            [explode_pos1[2], explode_pos2[2], explode_pos3[2]],
            c='orange', s=50, label='起爆点')

# Fake target and true target cylinder
ax1.scatter(0, 0, 0, c='black', s=100, label='假目标')
theta_cyl = np.linspace(0, 2 * np.pi, 50)
cyl_x = 7 * np.cos(theta_cyl)
cyl_y = 200 + 7 * np.sin(theta_cyl)
ax1.plot(cyl_x, cyl_y, zs=0, zdir='z', color='purple', label='真目标基底')
ax1.plot(cyl_x, cyl_y, zs=10, zdir='z', color='purple')
for i in range(0, len(theta_cyl), 10):
    ax1.plot([cyl_x[i], cyl_x[i]], [cyl_y[i], cyl_y[i]], [0, 10], color='purple', alpha=0.5)

ax1.scatter(FY1_INIT[0], FY1_INIT[1], FY1_INIT[2], c='blue', s=100, label='FY1 初始位置')
ax1.scatter(FY2_INIT[0], FY2_INIT[1], FY2_INIT[2], c='cyan', s=100, label='FY2 初始位置')
ax1.scatter(FY3_INIT[0], FY3_INIT[1], FY3_INIT[2], c='magenta', s=100, label='FY3 初始位置')
ax1.scatter(m1_init[0], m1_init[1], m1_init[2], c='red', s=100, label='M1 初始位置')

ax1.set_xlabel('X (米)')
ax1.set_ylabel('Y (米)')
ax1.set_zlabel('Z (米)')
ax1.set_title('运动轨迹三维示意图')
ax1.legend()
plt.savefig('trajectories.png')
plt.show()

# Visualization 2: Shielding time distribution schematic
fig2, ax2 = plt.subplots(figsize=(10, 6))
t_axis = np.linspace(0, M1_TIME, 1000)
shielding_status = np.zeros_like(t_axis)
for start, end in shielding_intervals:
    mask = (t_axis >= start) & (t_axis <= end)
    shielding_status[mask] = 1

ax2.fill_between(t_axis, 0, shielding_status, alpha=0.7, color='green', label='屏蔽时间段')
ax2.plot(t_axis, shielding_status, 'g-', linewidth=2)
ax2.set_ylim(-0.1, 1.1)
ax2.set_xlabel('时间 (秒)')
ax2.set_ylabel('屏蔽状态')
ax2.set_title('屏蔽时间分布示意图')
explode_times = [t_deploy1 + delay1, t_deploy2 + delay2, t_deploy3 + delay3]
for et in explode_times:
    ax2.axvline(x=et, color='orange', linestyle='--', alpha=0.5, label='起爆时间' if et == explode_times[0] else "")
ax2.legend()
plt.savefig('shielding_distribution.png')
plt.show()
# Visualization 3: Three-view diagram (Front, Side, Top views)
fig3, (ax_front, ax_side, ax_top) = plt.subplots(1, 3, figsize=(18, 6))

# Front view (X-Z plane)
ax_front.plot(m1_traj[:, 0], m1_traj[:, 2], 'r-', label='导弹 M1 轨迹', linewidth=2)
ax_front.plot(fy1_traj[:, 0], fy1_traj[:, 2], 'b-', label='无人机 FY1 轨迹', linewidth=2)
ax_front.plot(fy2_traj[:, 0], fy2_traj[:, 2], 'c-', label='无人机 FY2 轨迹', linewidth=2)
ax_front.plot(fy3_traj[:, 0], fy3_traj[:, 2], 'm-', label='无人机 FY3 轨迹', linewidth=2)
ax_front.plot(bomb_traj1[:, 0], bomb_traj1[:, 2], 'g--', label='干扰弹 1 轨迹', alpha=0.7)
ax_front.plot(bomb_traj2[:, 0], bomb_traj2[:, 2], 'g--', label='干扰弹 2 轨迹', alpha=0.7)
ax_front.plot(bomb_traj3[:, 0], bomb_traj3[:, 2], 'g--', label='干扰弹 3 轨迹', alpha=0.7)
ax_front.scatter([explode_pos1[0], explode_pos2[0], explode_pos3[0]],
                [explode_pos1[2], explode_pos2[2], explode_pos3[2]],
                c='orange', s=50, label='起爆点')
ax_front.scatter(FY1_INIT[0], FY1_INIT[2], c='blue', s=100, label='FY1 初始位置')
ax_front.scatter(FY2_INIT[0], FY2_INIT[2], c='cyan', s=100, label='FY2 初始位置')
ax_front.scatter(FY3_INIT[0], FY3_INIT[2], c='magenta', s=100, label='FY3 初始位置')
ax_front.scatter(m1_init[0], m1_init[2], c='red', s=100, label='M1 初始位置')
ax_front.scatter(0, 0, c='black', s=100, label='假目标')
ax_front.plot([7, 7], [0, 10], 'purple', label='真目标', alpha=0.5)
ax_front.plot([-7, -7], [0, 10], 'purple', alpha=0.5)
ax_front.set_xlabel('X (米)')
ax_front.set_ylabel('Z (米)')
ax_front.set_title('前视图 (X-Z)')
ax_front.grid(True)
ax_front.legend()

ModuleNotFoundError: No module named 'consts_and_funcs'